### **Multicopter design calculation**

In [1]:
import math
import numpy as np
import pandas as pd
import splinecloud_scipy as scsp
import matplotlib.pyplot as plt
from dataclasses import dataclass
%matplotlib inline 
from scipy.optimize import brentq
from scipy.optimize import fmin
from pprint import pprint
from copy import deepcopy
# from scipy.optimize import differential_evolution

### Load data from SplineCloud

In [2]:
from components_data import propellers, motors, battery_groups, Propeller, Motor, Battery

### Define Constants

Payload mass

In [3]:
m_payload = 1.5

Hovering time

In [4]:
t_hover = 0.5 # [hours]
t_hover_min = 0.4

Maximum discharge ratio

In [5]:
k_discharge = 0.2

Hovering throttle range

In [6]:
# hov_throttle_range = np.linspace(0.4, 0.6, 11)
hov_throttle_range = np.linspace(0.4, 0.6, 5)
hov_throttle_range

array([0.4 , 0.45, 0.5 , 0.55, 0.6 ])

Copter mass to the payload mass

In [7]:
km_copter = 3.5

Total guess mass of the quadcopter

In [8]:
m_total = (km_copter+1)*m_payload

Gravity acceleration

In [9]:
g = 9.81

Console length factor

In [10]:
k_console = 2.5

Number of motors

In [11]:
n_motor = 4

Single cell voltage

In [12]:
cell_voltage = 3.7

Required hovering thrust per motor

In [13]:
F_hover = m_total*g/n_motor
F_hover

16.554375

Frame console inner diameter ratio (for round section)

In [14]:
kd_console = 0.9

Console length ratio

In [15]:
kl_console = 2.5

Console strength safety factor

In [16]:
ks_console = 1.2

Dynamic load factor

In [17]:
ks_dyn = 3

Yield strength of the console material

In [18]:
sigma_max = 90*1e6

Material density

In [19]:
ro_console = 2700 # kg/m^3 (Aluminium)

### Design calculation

#### **Motor selection**

In [20]:
def select_motor(motors, f_hover, throttle_range):
    selected_motor = None
    thrust_diff = math.inf
    for hov_throttle in throttle_range:
        for motor in motors:
            thrust = motor.thrust_vs_throttle.eval(hov_throttle)*0.001*g
            diff = abs(thrust - f_hover)
            if thrust_diff > diff:
                thrust_diff = diff
                selected_motor = deepcopy(motor)
                selected_motor.hov_throttle = hov_throttle

    return selected_motor

In [21]:
motor = select_motor(motors, F_hover, hov_throttle_range)
motor.name

'u7v2_kv420_22.2v_15x5cf'

In [22]:
# hov_throttle

#### **Battery selection**

In [23]:
def select_battery(voltage, current):
    n_cells = round(voltage/cell_voltage)
    capacity = (n_motor * current * t_hover) / (1-k_discharge)
    
    selected_battery = None
    total_weight = math.inf
    n_batteries = range(1, n_motor)
    for batgroup in battery_groups.keys():
        for cellgroup in battery_groups[batgroup].keys():
            snum = int(cellgroup[0])
            if snum == int(n_cells):
                candidates = battery_groups[batgroup][cellgroup]
                for nbat in n_batteries:
                    c = 1000*capacity/nbat  
                    bcaps = [i for i in range(len(candidates.data.index)) if candidates.data.iloc[i,3] >= c]
                    if not bcaps:
                        continue
                    closest_ind = bcaps[0]
                    battery_row = candidates.data.iloc[closest_ind,:]
                    w = battery_row.iloc[-1]
                    if w*nbat < total_weight:
                        total_weight = w*nbat
                        selected_battery = Battery(battery_row, nbat)
    
    return selected_battery

In [24]:
battery = select_battery(motor.voltage, motor.hover_current)
battery

Battery(name=HP-G830C10000S6, capacity=10000, s=6 number=3, total_weight=3699, total_capacity=30000)

#### **Frame design calculation**

*Console with round section and rectangular central plate*

In [25]:
@dataclass
class RoundConsole:
    length: float
    dout: float
    section_area: float
    weight: float


def design_frame_console(f_hover, d_propeller):
    length = d_propeller * kl_console
    
    dout = np.cbrt((f_hover*length*32*ks_console*ks_dyn)/(sigma_max*math.pi*(1-kd_console**4)))
    if dout > 5*1e-3:
        dout = math.ceil(dout*1e3)*1e-3
    else:
        dout = 5*1e-3
    
    section_area = (math.pi*(dout**2)*(1-kd_console**2))/4
    weight = section_area * length * ro_console
    
    return RoundConsole(length, dout, section_area, weight)

In [26]:
console = design_frame_console(motor.hover_thrust*g*1e-3, motor.propeller.diameter*0.0254)
console

RoundConsole(length=0.9525, dout=0.027, section_area=0.00010878549961218052, weight=0.2797691086276253)

#### **Copter total mass recalculation**

In [27]:
m_copter = n_motor*(console.weight + motor.weight + motor.propeller.weight) + battery.total_weight
m_copter

5064.11907643451

#### Refined copter reduced weight

In [28]:
km_copter = m_copter*1e-3/m_payload
km_copter

3.3760793842896732

#### **Design optimization**

In [29]:
def design_iteration(km_copter_in, iteration=False):
    m_total = (km_copter_in + 1) * m_payload
    f_hover = m_total*g / n_motor
    
    motor = select_motor(motors, f_hover, hov_throttle_range)
    battery = select_battery(motor.voltage, motor.hover_current)
    console = design_frame_console(motor.hover_thrust*g*1e-3, motor.propeller.diameter*0.0254)
    
    if not all([motor, battery, console]):
        if iteration:
            return km_copter_in*10
        else:
            return
    
    m_copter = n_motor*(console.weight + motor.weight + motor.propeller.weight) + battery.total_weight
    
    km_copter_out = m_copter*1e-3/m_payload
    
    res = {'km_copter_in': km_copter_in, 
           'km_copter_out': km_copter_out,
           'km_copter_diff': km_copter_out - km_copter_in,
           'm_copter': m_copter,
           'motor': motor,
           'battery': battery,
           'console': console,
    }
    
    if iteration:
        return abs(km_copter_out - km_copter_in)
    else:
        return res

In [30]:
design_iteration(3.76)

#### **Find solutions in a range of initial guesses**

In [31]:
global_solutions = {}
for kmcop in np.linspace(1, 10, 19):
    converged = fmin(design_iteration, kmcop, args=(True,), full_output=True, disp=1)
    if converged[-1] == 0:
        km_sol = round(converged[0][0], 3)
        res = design_iteration(km_sol)
        print(res)
        if res and km_sol not in global_solutions.keys():
            global_solutions[km_sol] = res

Optimization terminated successfully.
         Current function value: 0.184923
         Iterations: 15
         Function evaluations: 30
{'km_copter_in': 1.09, 'km_copter_out': 1.274962221558561, 'km_copter_diff': 0.1849622215585609, 'm_copter': 1912.4433323378416, 'motor': Motor(name=u3_kv700_14.8v_12x4cf, voltage=14.8, kv=700, weight=128, propeller=Propeller(name=P12x4 Prop-2PCS/PAIR, diameter=12, width=4.0)), 'battery': Battery(name=HP-G830C5200S4, capacity=5200, s=4 number=3, total_weight=1284, total_capacity=15600), 'console': RoundConsole(length=0.7619999999999999, dout=0.019, section_area=5.3870460027430955e-05, weight=0.11083308446043644)}
Optimization terminated successfully.
         Current function value: 0.084892
         Iterations: 14
         Function evaluations: 29
{'km_copter_in': 1.594, 'km_copter_out': 1.6793740334993448, 'km_copter_diff': 0.08537403349934469, 'm_copter': 2519.061050249017, 'motor': Motor(name=u7v2_kv280_24v_18x6.1cf, voltage=24, kv=280, weight=29

In [32]:
pprint(global_solutions)

{1.09: {'battery': Battery(name=HP-G830C5200S4, capacity=5200, s=4 number=3, total_weight=1284, total_capacity=15600),
        'console': RoundConsole(length=0.7619999999999999,
                                dout=0.019,
                                section_area=5.3870460027430955e-05,
                                weight=0.11083308446043644),
        'km_copter_diff': 0.1849622215585609,
        'km_copter_in': 1.09,
        'km_copter_out': 1.274962221558561,
        'm_copter': 1912.4433323378416,
        'motor': Motor(name=u3_kv700_14.8v_12x4cf, voltage=14.8, kv=700, weight=128, propeller=Propeller(name=P12x4 Prop-2PCS/PAIR, diameter=12, width=4.0))},
 1.273: {'battery': Battery(name=HP-G830C5200S4, capacity=5200, s=4 number=3, total_weight=1284, total_capacity=15600),
         'console': RoundConsole(length=0.8255,
                                 dout=0.021,
                                 section_area=6.580851211107218e-05,
                                 weight=0.14667

#### **Design verification**

In [33]:
def verify_solution(solution):
    ## verify thrust is enough to hover
    km, sol = solution
    
    motor, battery, console = sol["motor"], sol["battery"], sol["console"]
    
    thrust_verified = False
    copter_mass = 1e-3*(battery.total_weight +
        n_motor*(console.weight + motor.weight + motor.propeller.weight)
    )
    
    total_copter_weight = (copter_mass + m_payload) * g
    thrust_vs_throttle = motor.thrust_vs_throttle

    max_throttle_data = max(thrust_vs_throttle.coeffs_x)
    min_throttle_data = min(thrust_vs_throttle.coeffs_x)
    
    max_hov_throttle = min(max(hov_throttle_range), max_throttle_data)
    min_hov_throttle = max(min(hov_throttle_range), min_throttle_data)
    
    max_copter_thrust = thrust_vs_throttle.eval(max_hov_throttle)*0.001*g*n_motor
    min_copter_thrust = thrust_vs_throttle.eval(min_hov_throttle)*0.001*g*n_motor
    
    if not (min_copter_thrust <= total_copter_weight <= max_copter_thrust):
        return
    
    ## verify hover time is sufficient
    
    ## find exact hover throttle
    throttle_root = lambda throt: thrust_vs_throttle.eval(throt)*0.001*g*n_motor - total_copter_weight
    hov_throttle = brentq(throttle_root, min_hov_throttle, max_hov_throttle)
    
    ## calculate hover time
    current = motor.current_vs_throttle.eval(hov_throttle)
    t_hov = battery.capacity*1e-3 * battery.number * (1-k_discharge) / (n_motor * current)

    if t_hov >= t_hover_min:
        solstat = {
            "solution": solution,
            "copter_mass": copter_mass,
            "total_copter_weight": total_copter_weight,
            "hov_throttle": hov_throttle,
            "t_hov": t_hov,
        }
    
        return solstat

**Verified global solutions**

In [34]:
verified_solutions = [s for s in map(verify_solution, global_solutions.items()) if s is not None]

#### **Optimal solution**

In [35]:
opt_sol = None
copter_mass = math.inf
for vs in verified_solutions:
    # pprint(s)
    if vs['copter_mass'] < copter_mass:
        copter_mass = vs['copter_mass']
        opt_sol = vs
opt_sol

{'solution': (1.273,
  {'km_copter_in': 1.273,
   'km_copter_out': 1.27345780613925,
   'km_copter_diff': 0.000457806139250172,
   'm_copter': 1910.186709208875,
   'motor': Motor(name=u3_kv700_14.8v_13x4.4cf, voltage=14.8, kv=700, weight=128, propeller=Propeller(name=P13x4.4 Prop-2PCS/PAIR, diameter=13, width=4.4)),
   'battery': Battery(name=HP-G830C5200S4, capacity=5200, s=4 number=3, total_weight=1284, total_capacity=15600),
   'console': RoundConsole(length=0.8255, dout=0.021, section_area=6.580851211107218e-05, weight=0.14667730221876324)}),
 'copter_mass': 1.9101867092088751,
 'total_copter_weight': 33.453931617339066,
 'hov_throttle': 0.5503309520877482,
 't_hov': 0.5202606936133145}